In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

# ML Project

## Headers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/fake-news/train.csv")
#test = pd.read_csv("test.csv")

## Spliting Dataset

In [ ]:
train = train.dropna()
#test = test.dropna()

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [ ]:
train.drop(train.index[5000:], inplace=True)

In [ ]:
y_train = train['label']
X_train = train.drop(['id','label'],axis=1)



In [ ]:

#X_test = test.drop(['id'],axis=1)

## Cleaning Dataset

In [ ]:
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [ ]:
print(y_train.info())
X_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 5000 entries, 0 to 5715
Series name: label
Non-Null Count  Dtype
--------------  -----
5000 non-null   int64
dtypes: int64(1)
memory usage: 78.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 5715
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   5000 non-null   object
 1   author  5000 non-null   object
 2   text    5000 non-null   object
dtypes: object(3)
memory usage: 156.2+ KB


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### removing HTML tags, special characters and punctuations, converting to lowercase, removing stop words, stemming the words and joining back every stemmed word back to a single string

In [ ]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub('<.*?>', '', text)
    # Remove special characters and punctuation
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Stem the tokens
    ps = PorterStemmer()
    stemmed_tokens = [ps.stem(token) for token in filtered_tokens]
    # Join the tokens back into a single string
    text = ' '.join(stemmed_tokens)
    return text

In [ ]:
X_train['title'] = X_train['title'].apply(clean_text)
X_train['author'] = X_train['author'].apply(clean_text)
X_train['text'] = X_train['text'].apply(clean_text)

'''
X_test['title'] = X_test['title'].apply(clean_text)
X_test['author'] = X_test['author'].apply(clean_text)
X_test['text'] = X_test['text'].apply(clean_text)
'''

"\nX_test['title'] = X_test['title'].apply(clean_text)\nX_test['author'] = X_test['author'].apply(clean_text)\nX_test['text'] = X_test['text'].apply(clean_text)\n"

### Vectorize the text data using TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the text data
tfidf = TfidfVectorizer()
title = tfidf.fit_transform(X_train['title'])
author = tfidf.fit_transform(X_train['author'])
text = tfidf.fit_transform(X_train['text'])

'''
title1 = tfidf.fit_transform(X_test['title'])
author1 = tfidf.fit_transform(X_test['author'])
text1 = tfidf.fit_transform(X_test['text'])
'''

"\ntitle1 = tfidf.fit_transform(X_test['title'])\nauthor1 = tfidf.fit_transform(X_test['author'])\ntext1 = tfidf.fit_transform(X_test['text'])\n"

In [ ]:
#concatenating every features to a single input 

import numpy as np

X_train = np.concatenate((title.toarray(), author.toarray(), text.toarray()), axis=1)

#X_test = np.concatenate((title1.toarray(), author1.toarray(), text1.toarray()), axis=1)

In [ ]:
print(X_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


##Splitting Of Dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=50)

##Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = logreg.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))

Accuracy : 0.994
Confusion Matrix :
 [[574   3]
 [  3 420]]
Classification Report 
:                precision    recall  f1-score   support

           0       0.99      0.99      0.99       577
           1       0.99      0.99      0.99       423

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000



##Multinomial Navie Bayes

In [1]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = mnb.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))

Accuracy : 0.939
Confusion Matrix :
 [[560  17]
 [ 44 379]]
Classification Report 
:                precision    recall  f1-score   support

           0       0.93      0.97      0.95       577
           1       0.96      0.90      0.93       423

    accuracy                           0.94      1000
   macro avg       0.94      0.93      0.94      1000
weighted avg       0.94      0.94      0.94      1000



## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()

DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = DT.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))

Accuracy : 0.977
Confusion Matrix :
 [[567  10]
 [ 13 410]]
Classification Report 
:                precision    recall  f1-score   support

           0       0.98      0.98      0.98       577
           1       0.98      0.97      0.97       423

    accuracy                           0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000



## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200,criterion='entropy', random_state=50)

rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200, random_state=50)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = rf.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))

Accuracy : 0.957
Confusion Matrix :
 [[571   6]
 [ 37 386]]
Classification Report 
:                precision    recall  f1-score   support

           0       0.94      0.99      0.96       577
           1       0.98      0.91      0.95       423

    accuracy                           0.96      1000
   macro avg       0.96      0.95      0.96      1000
weighted avg       0.96      0.96      0.96      1000



## SVM

In [ ]:
from sklearn import svm

sv = svm.SVC()
sv.fit(X_train, y_train)

SVC()

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = sv.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))

Accuracy : 0.997
Confusion Matrix :
 [[576   1]
 [  2 421]]
Classification Report 
:                precision    recall  f1-score   support

           0       1.00      1.00      1.00       577
           1       1.00      1.00      1.00       423

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



##AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ABC = AdaBoostClassifier( n_estimators=50, random_state=50)
ABC.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = ABC.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))

##GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
GBC = GradientBoostingClassifier( n_estimators=50, random_state=50)
GBC.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
y_pred = GBC.predict(X_test)


# Compute classification metrics
print("Accuracy :",accuracy_score(y_test, y_pred))
print ('Confusion Matrix :\n',confusion_matrix(y_test, y_pred))
print ('Classification Report \n: ',classification_report(y_test, y_pred))